# INF8111 - Fouille de données

## TP1 Automne 2020 - Duplicate Bug Report Detection

##### Membres de l'équipe / Team members:

    - Sabzi Dizajyekan (2078921) 1
    - Desclaux  (2097696) 2
    - Berrais-Sanchez  (2092882) 3
    



## 1 - Résumé / Overview

À cause de la complexité des systèmes informatiques, les bogues logiciels sont courants. Les entreprises, et en particulier les plus grosses, utilisent un *Bug Tracking System* (BTS), aussi appelé *Issue Tracking System*, pour organiser et suivre les rapports de bogues. En plus des développeurs et des testeurs, de nombreux projets (notamment ceux libres de droits) permettent aux utilisateurs de soumettre un rapport de bogues dans leur BTS. Pour ce faire, les utilisateurs doivent remplir un formulaire avec plusieurs champs. La majorité de ces champs contient des données catégoriques et accepte uniquement des valeurs prédéfinies (composant, version du produit et du système, etc.). Deux autres champs importants sont le résumé ou *summary* en anglais, et la description. Les utilisateurs sont libres d’écrire ce qu’ils veulent dans ces deux champs avec pour seule contrainte le nombre de caractères. La soumission de ces champs crée une page que l’on appelle rapport de bogue et qui contient toute l’information à propos du bogue.

Par manque de communication et de synchronisation, les utilisateurs ne savent pas toujours qu’un bogue a déjà été soumis et peuvent donc le soumettre à nouveau. Identifier les rapports qui correspondent au même bogue (duplicata) est une tache importante des BTSs et est le sujet de ce TP. Notre objectif est de développer un système qui comparera les nouveaux rapports de bogues avec ceux déjà soumis en fonction de leur similarité textuelle. La liste triée des rapports les plus similaires sera utilisée par un opérateur humain pour identifier manuellement si un rapport est un duplicata.

---

Due to the complexity of software systems, software bugs are prevalent. Companies, especially the larger ones, usually use a Bug Tracking System (BTS), also called Issue Tracking System, to manage and track records of bugs. Besides developers and testers, many projects, mainly open source projects, allow users to report new bugs in their BTS.
To do that, users have to fill out a form with multiple fields. An important subset of
these fields provides categorical data and only accepts values that range from a fixed list of
options (e.g. component, version and product of the system). Two other important fields
are the summary and the description. The users are free to write anything in both fields
with the only constraint that the summary has a maximum number of characters. The
submission of a form creates a page, called bug report or issue report which contains all
the information about a bug.

Due to the lack of communication and synchronization, users may not be aware that a specific bug was already submitted and may report it again. Identifying duplicate bug reports is an important task in the BTSs and is the subject of this TP. Our objective is to develop a system that will compare a new bug report with the already submitted ones and rank them based on textual similarity. This ranked list will be used by a triager to manually identify whether a report is a duplicate or not.


# 2 Installation / Setup

Pour ce TP, vous aurez besoin des librairies `numpy`, `sklearn` et `scipy` (que vous avez sans doute déjà), ainsi que la librairie `nltk`, qui est une libraire utilisée pour faire du traitement du language (Natural Language Processing, NLP). Installez les libraires en question et exécutez le code ci-dessous :

---

For this assignment, you need the `numpy`, `sklearn` and `scipy` libraries (which you may already have), as well as the `nltk` library, which is used for Natural Language Processing (NLP). Please run the code below to install the packages needed for this assignment.

In [ ]:
                                                                                                                                                                                                                # If you want, you can use anaconda and install after nltk library
# pip install --user numpy
# pip install --user sklearn
# pip install --user scipy
# pip install --user nltk
# pip install --user tqdm


#python
#import nltk

## 3 - Jeu de données / Data

Téléchargez l'archive à l'adresse suivante: https://drive.google.com/file/d/14BrFaOiVDIcpjwpm5iAkAy0v_moZyGiP/view?usp=sharing

In this zip file, there are: 

L'archive contient:
1. test.json: Ce fichier contient les duplicate bug reports that will be used to evaluate our system.
2. threads: Ce dossier contient le code HTML des bug report. Chaque fichier HTML est nommé selon le motif **bug_report_id.html**.


L'image ci-dessous illustre un exemple de bug report:

![https://ibb.co/tqyRM4L](bug_report.png)

- A : identifiant du bug report
- B : date de création
- C : résumé
- D : composant
- E : produit
- F : l'identifiant du rapport dont le bug report est dupliqué
- G : description


Le script suivant charge le jeu de données de test et définit certaines variables globales:

---

Please download the zip file at the following adress: https://drive.google.com/file/d/14BrFaOiVDIcpjwpm5iAkAy0v_moZyGiP/view?usp=sharing

This archive contains: 

1. test.json: This file contains duplicate bug reports that will be used to evaluate our system.
2. bug_reports: It is a folder that contains the bug report html source. Each html file name follows the pattern **bug_report_id.html**.


Figure below depicts an bug report page example:

![https://ibb.co/tqyRM4L](bug_report.png)


- A : bug report id
- B : creation date
- C : summary
- D : component
- E : product
- F : the report id which the bug report is duplicate
- G : description

 The following script loads the test dataset and define some global variables:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# define the folder path that contain the data
# FOLDER_PATH = "Define folder path that contain threads folder and test.json"
FOLDER_PATH = "/content/drive/My Drive/Colab Notebooks/Fouille de donnees/TP1/INF8111_2020_fall_tp1_dataset/dataset"
PAGE_FOLDER = os.path.join(FOLDER_PATH, 'bug_reports')



# Load the evaluation dataset
import json


test = json.load(open(os.path.join(FOLDER_PATH,'test.json')))

# 4 - Web scraping

"Le *web scraping* (parfois appelé harvesting) est une technique d'extraction du contenu de sites Web, via un script ou un programme, dans le but de le transformer pour permettre son utilisation dans un autre contexte, par exemple le référencement." [Wikipedia](https://fr.wikipedia.org/wiki/Web_scraping)

---

*Web scraping* also called harvesting consists in extracting relevant data from web pages and prepare it for computational analysis.


## 4.1 - Question 1 (2 points)

Implémentez *extract_data_from_page*. Cette fonction extrait l'information suivante du code HTML : identifiant du rapport de bogue, date de création, titre, produit, composant, identifiant du rapport de bogue bug dont il est un duplicata, résumé et description. 


La fonction *extract_data_from_page* retourne un dictionnaire avec la structure suivante :
```
 {"report_id": int, 
  "dup_id": int or None (the report id which it is duplicate), 
  "component": string, 
  "product": string, 
  "summary": str, 
  "description": string, 
  "creation_date": int} 
```

Par exemple, quand la fonction *extract_data_from_page* reçoit le rapport "bug_report/7526.html", elle retourne :

```
{
"report_id": 7526,
"dup_id": 7799,
"product": "core graveyard",
"component":  tracking,
"summary": "Apprunner crashes on exit",
"description": "Apprunner crashes on exit, without fail. The browser window closes, but the
console window hangs around. I end up having to kill winoldap with the \\"Close
Program\\" dialog (Ctrl-Alt-Del).",
"creation_date": 928396140
}
```

**La date de création doit être représentée comme un timestamp (entier). Si un bug n'est pas un duplicata, alors dup_id doit être None.**

*Indice: lxml parse est plus rapide que html.parser*

---

Implement *extract_data_from_page*. This function extracts the following information from the html: bug report id, creation date, title, product, component, the report id which it is duplicate, summary and description.

The *extract_data_from_page* function returns a dictionary with the following structure:
```
 {"report_id": int, 
  "dup_id": int or None (the report id which it is duplicate), 
  "component": string, 
  "product": string, 
  "summary": str, 
  "description": string, 
  "creation_date": int} 
```

For instance, when extract_data_from_page receives "bug_report/7526.html", it returns:

```
{
"report_id": 7526,
"dup_id": 7799,
"product": "core graveyard",
"component":  tracking,
"summary": "Apprunner crashes on exit",
"description": "Apprunner crashes on exit, without fail. The browser window closes, but the
console window hangs around. I end up having to kill winoldap with the \\"Close
Program\\" dialog (Ctrl-Alt-Del).",
"creation_date": 928396140
}
```

**Creation date have to be represented as timestamp. If bug report is not duplicate, dup_id have to be None.**

*HINT: lxml parse is faster than html.parser*


In [ ]:
!pip install bs4
from bs4 import BeautifulSoup
!pip install lxml
import re
# define the Dictionary report
def extract_data_from_page(pagepath):
    report={
        "report_id": int,
        "dup_id": int or None,
        "component": str,
        "product": str,
        "summary": str,
        "description": str,
        "creation_date": int
    }
    with open(pagepath,"r",  encoding="UTF-8") as fp:
        soup = BeautifulSoup(fp, "lxml")
    
    report["report_id"]=int(re.search(r"[0-9]+",soup.find("span", {"id":"field-value-bug_id"}).getText()).group())

    report["creation_date"]=int(soup.find("span",{"class":"rel-time", "data-time":True})["data-time"])

    report["summary"]=soup.find("h1", {"id":"field-value-short_desc"}).getText()


    report["component"] = "".join(soup.find("span", {"id": "component-name"}).getText().splitlines())


    report["product"] = "".join(soup.find("span", {"id": "product-name"}).getText().splitlines())

    dup_id=soup.find("span",{"id":"field-value-status-view"})
    if dup_id.find('a'):
      report["dup_id"]=int(re.search(r"[0-9]+",dup_id.find('a').getText()).group())
    else:
      report["dup_id"]=None
    
    report['description']=soup.find("pre", {"class":"comment-text"}).getText()
    return(report)

    


## 4.3 - Extraire le texte du code HTML / Extract text from HTML


In [ ]:
import os
from multiprocessing import Pool, TimeoutError
from time import time
import json
import tqdm

# Index each thread by its id
index_path = os.path.join(PAGE_FOLDER, 'bug_reports.json')

if os.path.isfile(index_path):
    # Load threads that webpage content were already extracted.
    report_index = json.load(open(index_path))
else:
    # Extract webpage content

    # This can be slow (around 10 minutes). Test your code with a small sample. lxml parse is faster than html.parser
    files = [os.path.join(PAGE_FOLDER, filename) for filename in os.listdir(PAGE_FOLDER)]
    reports = [extract_data_from_page(f) for f in tqdm.tqdm(files)]
    report_index = dict(((report['report_id'], report) for report in reports ))

    # Save preprocessed bug reports
    json.dump(report_index, open(index_path,'w'))
    

100%|██████████| 9998/9998 [09:25<00:00, 17.69it/s]


In [ ]:
index_path = os.path.join(PAGE_FOLDER, 'bug_reports.json')
report_index = json.load(open(index_path))




# 5 - Prétraitement des données / Data Preprocessing

Le prétraitement des données est une tache cruciale en fouille de données. Cette étape nettoie et transforme les données brutes dans un format qui permet leur analyse, et leur utilisation avec des algorithmes de *machine learning*. En traitement des langages (natural language processing, NLP), la *tokenization* et le *stemming* sont des étapes cruciales. De plus, vous implémenterez une étape supplémentaire pour filtrer les mots sans importance.

---

Preprocessing is a crucial task in data mining. The objective is to clean and transform the raw data in a format that is better suited for data analysis and machine learning techniques. In natural language processing (NLP), *tokenization* and *stemming* are two well-known preprocessing steps. Besides these two steps, we will implement an additional step that is designed exclusively for the twitter domain.

## 5.1 - Tokenization

Cette étape permet de séparer un texte en séquence de *tokens* (= jetons, ici des mots, symboles ou ponctuation). Par example, la phrase *"It's the student's notebook."* peut être séparé en liste de tokens de cette manière: ["it", " 's", "the", "student", " 's", "notebook", "."].

---

In this preprocessing step, a *tokenizer* is responsible for breaking a text in a sequence of tokens (words, symbols, and punctuation). For instance, the sentence *"It's the student's notebook."* can be split into the following list of tokens: ['It', "'s", 'the', 'student', "'s", 'notebook', '.'].


### 5.1.1 - Question 2 (0.5 points) 
Implémentez la fonction suivante :

- **tokenize_space** qui tokenize le texte à partir des blancs (espace, tabulation, nouvelle ligne). Ce tokenizer est naïf.
- **tokenize_nltk** qui utilise le tokenizer par défaut de la librairie nltk (https://www.nltk.org/api/nltk.html).
- **tokenize_space_punk** replace la ponctuation par des espaces puis tokenize les tokens qui sont séparés par des blancs (espace, tabulation, retour à la ligne).

**Tous les tokenizers doivent mettre les tokens en minuscule.**

---

Implement the following functions: 
- **tokenize_space** tokenizes the tokens that are separated by whitespace (space, tab, newline). This is a naive tokenizer.
- **tokenize_nltk** uses the default method of the nltk package (https://www.nltk.org/api/nltk.html) to tokenize the text.
- **tokenize_space_punk** replaces the punctuation to space and then tokenizes the tokens that are separated by whitespace (space, tab, newline).

**All tokenizers have to lowercase the tokens.**


In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenize_space(text):
    return (text.lower().split())


def tokenize_nltk(text):
    return (word_tokenize(text.lower()))


def tokenize_space_punk(text):
    return (re.split(r'[\s\t\n]+',re.sub(r'\W',' ',text.lower())))




    
        

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 5.2 - Suppression des mots vides / Stop words removal

### 5.2.1 - Question 3 (0.5 point)

Certains tokens sont sans importance pour la comparaison, car ils apparaissent dans la majorité des discussions. Les supprimer réduit la dimension du vecteur et accélère les calculs.

Expliquez quels tokens sont sans importances pour la comparaison des discussions. Implémentez la fonction filter_tokens qui retire ces mots de la liste des tokens.

---

There are a set of tokens that are not significant to the similarity comparison since they appear in most of bug report pages. Thus, removing them decreases the vector dimensionality and turns the similarity calculation computationally cheaper.

Describe the tokens that can be removed without affecting the similarity comparison? Moreover, implement the function filter_tokens that removes these words from a list of tokens.


# **The words that can be removed from a text that will not affect the similarity comparison are called stop words. The updated list of stop words are a list of 179 words which mainly include pronouns, auxilliary verbs, possessive adjectives and pronouns, preposition, conjunctions, adverbs, modal verbs and their negation. these kinds of words do not add new information to the text. Thereofore, in similarity comparison they do not have any value. In addition, since these words are very common and repititive, removing these words will significantly decrease  both computation time and storage volume. **


> Indented block



In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
def filter_tokens(tokens):
    stop_words = set(stopwords.words("english"))
    return([token for token in tokens if token not in stop_words])




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 5.3 - Racinisation / Stemming

La racinisation, ou *stemming* en anglais, est un procédé de transformation des flexions en leur radical ou racine. Par exemple, en anglais, la racinisation de "fishing", "fished" and "fish" donne "fish" (stem). 

---

The process to convert tokens with the same stem (word reduction that keeps word prefixes) to a standard form is called *stemming*. For instance, the word "fishing", "fished" , "fish" and "fisher" are reduced to the stem "fish".



In [ ]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')


word1 = ['I', 'tried', 'different', 'fishes']

print([ stemmer.stem(w) for w in word1])

word2 = ['I', 'will', 'tries', 'only', 'one', 'fishing']
print([ stemmer.stem(w) for w in word2])

['i', 'tri', 'differ', 'fish']
['i', 'will', 'tri', 'onli', 'one', 'fish']


### 5.3.1 - Question 4 (0.5 point) 

Expliquez comment et pourquoi le stemming est utile à notre système.

---

Explain how stemming can benefit our system?


# **The stemming will reduce the words to their stem and map several derivative of a word to the original stem. This step in pre-processing decrease the number of words in the text and create common entries between texts which will lead to higher similarity between texts.**
**bold text**# New Section







# 6 - Représentation des données / Data Representation

# 6.1 - Bag of Words

De nombreux algorithmes demandent des entrées qui soient toutes de la même taille, ce qui n'est forcément le cas pour des types de données comme les textes, qui peuvent avoir un nombre variable de mots.  

Par exemple, considérons la phrase 1, ”Board games are much better than video games” et la phrase 2, ”Monopoly is an awesome game!”. La table ci-dessous montre un exemple d'un moyen de représentation de ces deux phrases en utilisant une représentation fixe : 

|<i></i>     | an | are | ! | Monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 1 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |

Chaque colonne représente un mot du vocabulaire (de longueur 13), tandis que chaque ligne contient l'occurrence des mots dans une phrase. Ainsi, la valeur 2 à la position (1,7) est due au fait que le mot *"games"* apparaît deux fois dans la phrase 1. 

Ainsi, chaque ligne étant de longueur 13, on peut les utiliser comme vecteur pour représenter les phrases 1 et 2. Ainsi, c'est cette méthode que l'on appelle *Bag-of-Words* : c'est une représentation de documents par des vecteurs dont la dimension est égale à la taille du vocabulaire, et qui est construite en comptant le nombre d'occurrences de chaque mot. Ainsi, chaque token est ici associé à une dimension.

---

Many algorithms only accept inputs that have the same size. However, there are some data types whose sizes are not fixed, for instance, a text can have an unlimited number of words. Imagine that we retrieve two tweets: ”Board games are much better than video games” and ”Monopoly is an awesome game!”. These sentences are respectively named as Sentences 1 and 2. The table below depicts how we could represent both sentences using a fixed representation.

|            | an | are | ! | monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 0 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |

Each column of this table 2.1 represents one of 13 vocabulary words, whereas the rows contains the word
frequencies in each sentence. For instance, the cell in row 1 and column 7 has the value 2
because the word games occurs twice in Sentence 1. Since the rows have always 13 values, we
could use those vectors to represent the sentences 1 and 2. The table above illustrates a technique called bag-of-words. Bag-of-words represents a document as a vector whose dimensions are equal to the number of times that vocabulary words appeared in the document. Thus, each token will be related to a dimension, i.e. an integer.

<!-- Using raw frequency in the bag-of-words can be problematic. The word frequency distribution
is skewed - only a few words have high frequencies in a document. Consequently, the
weight of these words will be much bigger than the other ones which can give them more
impact on some tasks, like similarity comparison. Besides that, a set of words (including
those with high frequency) appears in most of the documents and, therefore, they do not
help to discriminate documents. For instance, the word *of* appears in a significant
number of tweets. Thus, having the word *of* does not make
documents more or less similar. However, the word *terrible* is rarer and documents that
have this word are more likely to be negative. TF-IDF is a technique that overcomes the word frequency disadvantages. -->

### 6.1.2 - Question 5 (2 points)


Implémentez le Bag-of-Words en pondérant le vecteur par la fréquence de chaque mot.

**Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe [sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html) de scipy.**

---

Implement the bag-of-words model that weights the vector with the absolute word frequency.

**For this exercise, you cannot use any external python library (e.g. scikit-learn). However, if you have a problem with memory size, you can use the scipy class [sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).**



In [ ]:
from scipy.sparse import csr_matrix
def transform_count_bow(X):
    data=[]
    indptr = [0]
    indices = []
    bag_words = {}
    i=0
    for sentence in X:
        for token in sentence:
            if token not in bag_words:
                bag_words[token]=i
                i+=1
    for sentence in X:
        tf={}
        for token in sentence:
            if token not in tf:
                tf[token]=1
            elif token in tf:
                tf[token]+=1
        
        for word in tf:
            data.append(tf[word])
            index = bag_words[word]
            indices.append(index)
        indptr.append(len(indices))
    sentence_matrix=csr_matrix((data, indices, indptr), dtype=int)
    
            
   
    return (sentence_matrix)

                   


## 6.2 - TF-IDF

L'utilisation de la fréquence d'apparition brute des mots, comme c'est le cas avec le bag-of-words, peut être problématique. En effet, peu de tokens auront une fréquence très élevée dans un document, et de ce fait, le poids de ces mots sera beaucoup plus grand que les autres, ce qui aura tendance à biaiser l'ensemble des poids. De plus, les mots qui apparaissent dans la plupart des documents n'aident pas à les discriminer. Par exemple, le mot "*de*" apparaît dans beaucoup de documents de la base de données, et pour autant, avoir ce mot en commun ne permet pas de conclure que des documents sont similaires. Au contraire, le mot "*génial*" est plus rare, mais les documents qui contiennent ce mot sont plus susceptibles d'être positifs. TF-IDF est donc une méthode qui permet de pallier à ce problème.

TF-IDF pondère le vecteur en utilisant une fréquence de document inverse (IDF) et une fréquence de termes (TF).

TF est l'information locale sur l'importance qu'a un mot dans un document donné, tandis que IDF mesure la capacité de discrimination des mots dans un jeu de données. 

L'IDF d'un mot se calcule de la façon suivante:

\begin{equation}
  \text{idf}_i = \log\left( \frac{N}{\text{df}_i} \right),
\end{equation}

Avec $N$ le nombre de documents dans la base de données, et $\text{df}_i$ le nombre de documents qui contiennent le mot $i$.

Le nouveau poids $w_{ij}$ d'un mot $i$ dans un document $j$ peut ensuite être calculé de la façon suivante:

\begin{equation}
  w_{ij} = \text{tf}_{ij} \times \text{idf}_i,
\end{equation}

avec $\text{tf}_{ij}$ la fréquence du mot $i$ dans le document $j$

---

Using raw frequency in the bag-of-words can be problematic. The word frequency distribution
is skewed - only a few words have high frequencies in a document. Consequently, the
weight of these words will be much bigger than the other ones which can give them more
impact on some tasks, like similarity comparison. Besides that, a set of words (including
those with high frequency) appears in most of the documents and, therefore, they do not
help to discriminate documents. For instance, the word *of* appears in a significant
number of tweets. Thus, having the word *of* does not make
documents more or less similar. However, the word *aeroplane* is rarer and documents that
have this word are more likely to be similar. TF-IDF is a technique that overcomes the word frequency disadvantages.

TF-IDF weights the vector using inverse document frequency (IDF) and word frequency, called term frequency (TF).
TF is the local information about how important is a word to a specific document.  IDF measures the discrimination level of the words in a dataset.  Common words in a domain are not helpful to discriminate documents since most of them contain these terms. So, to reduce their relevance in the documents, these words should have low weights in the vectors. 
The following equation computes the word IDF:
\begin{equation}
  idf_i = \log\left( \frac{N}{df_i} \right),
\end{equation}
Where $N$ is the number of documents in the dataset $df_i$ is the number of documents that contain a word $i$.
The new weight $w_{ij}$ of a word $i$ in a document $j$ using TF-IDF is computed as:
\begin{equation}
  w_{ij} = tf_{ij} \times idf_i,
\end{equation}
Where $tf_{ij}$ is the term frequency of words $i$ in the document $j$.



### 6.2.1 - Question 6 (2.5 points)

Implémentez le bag-of-words avec la pondération de TF-IDF

**Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe [sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html) de scipy.**

---

Implement a bag-of-words model that weights the vector using TF-IDF.

**For this exercise, you cannot use any external python library (e.g. scikit-learn). However, if you have a problem with memory size, you can use the scipy class [sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html)**



In [ ]:
import math
from scipy.sparse import csr_matrix
def transform_tf_idf_bow(X):
    bag_words={}
    IDF={}  
    i=0
    t={}
    indptr = [0]
    indices = []
    data = []
    
    for sentence in X:
        word_list=[]
        for token in sentence:
            if token not in word_list:
                word_list.append(token)
                if token not in bag_words:
                    bag_words[token]=i
                    i+=1
                    t[token]=1
                    
                elif token in bag_words:
                    t[token]+=1
                IDF[token]=math.log((len(X)/t[token]))
    
    for sentence in X:
        tf={}
        for token in sentence:
            if token not in tf:
                tf[token]=1
            elif token in tf:
                tf[token]+=1
        
        for word in tf:
            data.append(tf[word]*IDF[word])
            index = bag_words[word]
            indices.append(index)
        indptr.append(len(indices))
    sentence_matrix=csr_matrix((data, indices, indptr))      

    return (sentence_matrix)
        

# 7 - Notre système / Our System

## 7.1 - Question 7 (1 point)

La *pipeline* est la séquence d'étapes de prétraitement des données qui transforme les données brutes dans un format qui permet leur analyse. Pour notre problème, implémentez un pipeline composé des étapes suivantes :

1. Concatène le texte du résumé et description
2. Tokenize le texte, retire les stop words et stem le tokens. 
3. Génère la représentation vectorielle avec transform_tf_idf_bow ou transform_count_bow.
4. Encode données catégorielles (produit et composant) en entier 

La pipeline (fonction nlp_pipeline) prend en entrée la liste des rapports de bogues (liste des dictionnaires qui contiennent les informations des rapports), le type de tokenization, le type de vectorizer, un booléen qui active ou désactive la suppression des tokens inutiles et un booléen qui active ou désactive le stemming. La fonction nlp_pipeline retourne un tuple ($p$, $c$, $M$) :
- $p$ est le vecteur qui contient l'identifiant des produits des rapports de bogues
- $c$ est le vecteur qui contient l'identifiant des composants des rapports de bogues
- $M$ est une matrice avec la représentation du texte.

Le i-ème élément de $p$, $c$ et $M$ sont le produit, composant et la représentation du texte du i-ème rapport de bogue.


---

The pipeline is a sequence of preprocessing steps that transform the raw data to a format that is suitable for your problem. For our problem, you have to implement a pipeline that:

1. Concatenate the summary and description
2. Perform the tokenization, stop word removal and stemming in textual data
3. Generate the vector representation using transform_tf_idf_bow or transform_count_bow
4. Encode the categorical data (the component and product) to integers


The pipeline (nlp_pipeline function) receives a list of bug reports (dictionary that contains the report information), tokenization type, vectorizer type, a flag that enables or disable the insignificant token removal and a flag that turn stemming on or off. The nlp_pipeline function returns a tuple ($p$, $c$, $M$):
- $p$ is a vector that contains the product values of the bug reports
- $c$ is a vector that contains the component values of the bug reports
- $M$ is a matrix with the text representation.

The $i$-th element of $p$, $c$ and $M$ are the product, component and text representation of the $i$-th report in bug_reports, respectively.



In [ ]:
def nlp_pipeline(bug_reports, tokenization_type, vectorizer_type, enable_stop_words, enable_stemming):
  M=[]
  c=[]
  p=[]
  component_dict={}
  product_dict={}
  i=0
  j=0
  tokens_list=[]
  bug_reports_list=[]
  for bug_report in bug_reports:
    bug_reports_list.append(" ".join([bug_report['summary'],bug_report['description']]))
    if bug_report['component'] not in component_dict:
      component_dict[bug_report['component']]=i
      c.append(i)
      i+=1
    elif bug_report['component'] in component_dict:
      c.append(component_dict[bug_report['component']])
    if bug_report['product'] not in product_dict:
      product_dict[bug_report['product']]=j
      p.append(j)
      j+=1
    elif bug_report['product']  in product_dict:
      p.append(product_dict[bug_report['product']])
  for report in bug_reports_list:
    if tokenization_type is "space_tokenization":
      tokens=tokenize_space(report) 
    elif tokenization_type is "nltk_tokenization":
      tokens=tokenize_nltk(report)
    elif tokenization_type is "space_punk_tokenization":
      tokens=tokenize_space_punk(report)
    if enable_stop_words is True:
      tokens=filter_tokens(tokens)
   
    if enable_stemming is True:
      tokens=[stemmer.stem(token) for token in tokens]
    
    tokens_list.append(tokens)
  
  if vectorizer_type is "count":
    M=transform_count_bow(tokens_list)
  elif vectorizer_type is "tf_idf":
    M=transform_tf_idf_bow(tokens_list)
  
  return(p,c,M)




## 7.2 - Question 8 (1 point)

Implémentez la fonction rank qui retourne la liste des indices des rapports de bogues triée en fonction de la similarité entre les rapports de bug (candidat) et du nouveau rapport (requête). Vous utiliserez la fonction de similarité suivante pour comparer deux rapports :

$$
\mathrm{SIM}(q,r) = w_1 * F_1(q,r) + w_2 * F_c(q,r) + w_3 * cos\_sim(\mathrm{txt}_q, \mathrm{txt}_c),
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{si } p_q= p_r\\
    0,              & \text{autrement},
\end{cases}
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{si } c_q = c_r\\
    0,              & \text{autrement},
\end{cases}
$$

Où $c_q$ et $c_r$ sont les composants de la requête et du candidat,
 $p_q$ et $p_r$ sont les produits de la requête et du candidat,
 $\mathrm{txt}_q$ et $\mathrm{txt}_c$ sont les représentations du texte de la requête et du candida. Les paramètres 
 w_1, w_2 et w_3 doivent être réglés.
 

**Pour cette question, la requête doit être retirée de la liste triée (sortie de la fonction rank).**

*Pour de meilleures performances, vous pouvez utiliser la fonction cos d'une libraire (par exemple [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)) pour calculer la similarité. Il est préférable de faire des opérations matricielles.*

---

Implement the function rank that returns a list of reports indexes sorted by similarity of the bug reports (candidates) and new bug report (query). We use the following similarity function to compare two bug reports:

$$
\mathrm{SIM}(q,r) = w_1 * F_1(q,r) + w_2 * F_c(q,r) + w_3 * cos\_sim(\mathrm{txt}_q, \mathrm{txt}_c),
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{if } p_q= p_r\\
    0,              & \text{otherwise},
\end{cases}
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{if } c_q = c_r\\
    0,              & \text{otherwise},
\end{cases}
$$
Where $c_q$ and $c_r$ are the query and candidate components,
 $p_q$ and $p_r$ are the query and candidate products,
 $\mathrm{txt}_q$ and $\mathrm{txt}_c$ are the query and candidate textual representations, respectively. The parameters 
 w_1, w_2 and w_3 are to be tuned.
 

**In this question, the query has to  be removed in the sorted list (rank output).**

*For better performance, you can use the cosine similarity from a library (e.g. [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)). Also, we recommend performing matrix operations.*

In [ ]:
from sklearn.metrics.pairwise  import cosine_similarity
import numpy as np
def rank(query_idx, p, c, M, w1, w2, w3):
  p_ar=np.array(p)
  c_ar=np.array(c)
  
  cos_sim=np.transpose(cosine_similarity(M,M[query_idx:query_idx+1]))
  FP=np.where(p_ar==p_ar[query_idx],1,0)
  FC=np.where(c_ar==c_ar[query_idx],1,0)
     
  sim=np.argsort(w1*FP+w2*FC+w3*cos_sim)
  return (sim[sim!=query_idx][::-1])
 
  
  



## 7.3 - Évaluation / Evaluation

Vous allez tester différentes configurations du système de recommandations. Ces configurations seront comparées avec la [mean average precision (MAP) metric](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision). Plus les discussions pertinentes sont recommandées rapidement (c.-à-d. en haut de la liste), plus élevé sera le score MAP. Ressources supplémentaires pour comprendre MAP: [recall and precision over ranks](https://youtu.be/H7oAofuZjjE) et [MAP](https://youtu.be/pM6DJ0ZZee0).


La fonction *eval* évalue une configuration spécifique du système

---

We will test different configurations of our recommender system. These configurations are compared using the [mean average precision (MAP) metric](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision). Basically, the closer relevant threads are from ranked list begining, the higher MAP is. Additional materials to undertand MAP: [recall and precision over ranks](https://youtu.be/H7oAofuZjjE) and [MAP](https://youtu.be/pM6DJ0ZZee0).


The function *eval* evaluates a specific configurantion of our system



In [ ]:
from statistics import mean 


def calculate_map(x):
    res = 0.0
    n = 0.0
    
    for query_id, corrects, candidate_ids in x:
        precisions = []
        for k, candidate_id in enumerate(candidate_ids):
            
            if candidate_id in corrects:
                prec_at_k = (len(precisions) + 1)/(k+1)
                precisions.append(prec_at_k)
                
            if len(precisions) == len(corrects):
                break
                            
        res += mean(precisions)
        n += 1
    
    return res/n
            

def eval(tokenization_type, vectorizer, enable_stop_words, enable_stemming, w1, w2, w3):
    reports = [r for r in report_index.values()]
    report_ids = [r["report_id"] for r in report_index.values()]
    prod_v, comp_v, M = nlp_pipeline(reports, tokenization_type, vectorizer, enable_stop_words, enable_stemming)
    report2idx = dict([(r['report_id'], idx) for idx,r in enumerate(reports)])
    rank_lists = []
    for query_id, corrects in test:
        query_idx =  report_ids.index(query_id)
        candidate_idxs = rank(query_idx, prod_v, comp_v, M, w1, w2, w3)
        candidate_ids = [ report_ids[idx] for idx in candidate_idxs]
                
        rank_lists.append((query_id, corrects, candidate_ids))
        
        
    return calculate_map(rank_lists)

#eval( "space_tokenization", "count", False, False, 1, 1, 7)   
#eval( "nltk_tokenization", "count", False, False, 1, 1, 7)
#eval( "space_punk_tokenization", "count", False, False, 0.7, 0.7, 5)
#eval( "space_punk_tokenization", "count", True, False, .7, .7, 8)
#eval( "space_punk_tokenization", "count", True, True, 0.8, 0.8, 10)
#eval( "space_punk_tokenization", "tf_idf", False, False, 5, 6, 100)
#eval( "space_punk_tokenization", "tf_idf", True, False, 7, 7, 152)
eval( "space_punk_tokenization", "tf_idf", True, True, 7, 7, 152)

    

0.2208218424059815

## 7.4 - Question 9 (4 points)

Évaluez votre système avec chacune des configurations suivantes :

1. count(BoW) + space_tokenization
2. count(BoW) + nltk_tokenization
2. count(BoW) + space_punk_tokenization
3. count(BoW) + space_punk_tokenization + Stop words removal
4. count(BoW) + space_punk_tokenization + Stop words removal + Stemming
5. tf_idf + space_punk_tokenization
6. tf_idf + space_punk_tokenization + Stop words removal
7. tf_idf + space_punk_tokenization + Stop words removal + Stemming 

**Pour chaque configuration :** 

1. Rapportez la performance avec $w_1=0$ et $w_2=0$

2. Réglez les paramètres $w_1$, $w_2$ et $w_3$ et rapportez les valeurs qui donne les 3 meilleurs et les 3 pires résultats.

**En plus, décrivez et comparez vos résultats. Répondez aux questions suivantes :**

- Quelle méthode de tokenization donne les meilleurs résultats ? À votre avis, pourquoi ?
- Les étapes de prétraitement ont-elles un impact positif ou négatif sur notre système ?
- Est-ce que TF-IDF permet d'obtenir de meilleures performances que CountBoW? Si oui, à votre avis, pourquoi ?
- Est-ce que la comparaison du composant et du produit affecte positivement notre méthode ? 

**Notez qu'il y a une valeur minimum de  MAP à atteindre pour chaque configuration en dessous de laquelle la question sera pénalisée de 50%.**

1. count(BoW) + space_tokenization: 0.090 
2. count(BoW) + nltk_tokenization: 0.090
2. count(BoW) + space_punk_tokenization: 0.120
3. count(BoW) + space_punk_tokenization + Stop words removal: 0.170
4. count(BoW) + space_punk_tokenization + Stop words removal + Stemming: 0.195
5. tf_idf + space_punk_tokenization: 0.210
6. tf_idf + space_punk_tokenization + Stop words removal: 0.210
7. tf_idf + space_punk_tokenization + Stop words removal + Stemming: 0.215

---

Evaluate the system using each one of the following configurations:

1. count(BoW) + space_tokenization
2. count(BoW) + nltk_tokenization
2. count(BoW) + space_punk_tokenization
3. count(BoW) + space_punk_tokenization + Stop words removal
4. count(BoW) + space_punk_tokenization + Stop words removal + Stemming
5. tf_idf + space_punk_tokenization
6. tf_idf + space_punk_tokenization + Stop words removal
7. tf_idf + space_punk_tokenization + Stop words removal + Stemming 

**For each configuration:** 

1. Report the method performance achieved when $w_1=0$ and $w_2=0$

2. Tune the parameters $w_1$, $w_2$ and $w_3$ and report the parameter values that achieve the 3 best and 3 worst results.

**Also, describe and compare the results found by you and answer the following questions:**

- Which tokenization strategy has achieved the best result? Why do you think this has occurred?
- Was our system negatively or positively impacted by data preprocessing steps?
- TF-IDF has achieved a better performance than CountBoW? If yes, why do you think that this has occurred?
- Did the component and product comparison positively affect our method? 

**Note that there is a minimum MAP value to achieve for each configuration (see below) below which the question will be penalized by 50%.**

1. count(BoW) + space_tokenization: 0.090 
2. count(BoW) + nltk_tokenization: 0.090
2. count(BoW) + space_punk_tokenization: 0.120
3. count(BoW) + space_punk_tokenization + Stop words removal: 0.170
4. count(BoW) + space_punk_tokenization + Stop words removal + Stemming: 0.195
5. tf_idf + space_punk_tokenization: 0.210
6. tf_idf + space_punk_tokenization + Stop words removal: 0.210
7. tf_idf + space_punk_tokenization + Stop words removal + Stemming: 0.215

 *Which tokenization strategy has achieved the best result? Why do you think this has occurred?*
**space_punk_tokenization" gives the best similarity score.  Because it removes punctuations, links, etc,  therefore the similarity of texts increases. In "space_tokenization" pontuations adhered to the words that causes to have different words for a single word. for instance, "(size, size., 'size, "size", size" are all the same but "space_tokenization" produce different tokens for that single word. The "nltk_tokenization" consider punctuations and symbols (like @,#) as a token and create false similarity while they add no information for similarity. On the other hand, "space_punk_tokenization" removes punctuations, paths, links and etc. therefore keeps tokens that add information to the text. In addition, "space_punk_tokenization" produces less tokens that need less computation and storage resources.** 


*Was our system negatively or positively impacted by data preprocessing steps?* 
**Preprocessing steps are crucial and necessary for every analysis. The preprocessing methods removes unnecessary texts and transform textual data into numerical variable that is suitable for machine learning algorithms. These steps are crucial for similarity comparisons. Data of about 10,000 HTML files are extracted in less than 10 minutes and information is analyzed and sorted in about 1 minute which makes the future analysis very easy.**


*TF-IDF has achieved a better performance than CountBoW? If yes, why do you think that this has occurred?* 
**Yes TF-IDF has better performance than "Count" approach. While "count" only returns the frequency of a word in a text, "TF_IDF" measure relevance of a word in whole dataset.In other words this approach measure importance of words in a dataset (in constrast to "count" that measure frequency of words in the sentence. This is very useful to find similarity between sentences in a dataset.**

*Did the component and product comparison positively affect our method?* **Yes, incorporating product and component improve MAP value which shows comparison of these two entries improve ouput of recommendation system. unlike "summary" and "description" section, these two sections have limited entries ("component" and "product" should be selected from drop-down list by customer), therefore similarity between tokens are higher that makes these two sections very effective in similarity comparison. This is evidenty can be seen when comparing MAP value when w1=w2=0 with other cases that these weights are non-zero.**

```
eval( "space_tokenization", "count", False, False, 0, 0, 1)=0.0838803596072975   
eval( "nltk_tokenization", "count", False, False, 0, 0, 1)=0.0818958631605452
eval( "space_punk_tokenization", "count", False, False, 0, 0, 1)=0.11539725767824005
eval( "space_punk_tokenization", "count", True, False, 0, 0, 1)=0.16943314296473047
eval( "space_punk_tokenization", "count", True, True, 0, 0, 1)=0.19874981707562836
eval( "space_punk_tokenization", "tf_idf", False, False, 0, 0, 1)=0.20327563980859167
eval( "space_punk_tokenization", "tf_idf", True, False, 0, 0, 1)=0.20169687363315852
eval( "space_punk_tokenization", "tf_idf", True, True, 0, 0, 1)=0.21078433195606228 
```



```
eval( "space_tokenization", "count", False, False, 0.14, 0.13, 6) =0.09018535103352153
eval( "space_tokenization", "count", False, False, 0.13, 0.14, 6)=0.09022904411303871
eval( "space_tokenization", "count", False, False, 0.12, 0.14, 6)=0.09007635984918488

eval( "space_tokenization", "count", False, False, 1, 1, 7)=0.09802229146823004
eval( "space_tokenization", "count", False, False, 1, 1, 8)=0.09854910909309975
eval( "space_tokenization", "count", False, False, 1, 1, 9)=0.09828799823854747
```





```
eval( "nltk_tokenization", "count", False, False, 0.18, 0.19, 6)= 0.09083751108718754
eval( "nltk_tokenization", "count", False, False, 0.17, .19, 6)=0.0908727356136755
eval( "nltk_tokenization", "count", False, False, 0.175, .19, 6)=0.09080823508139146

eval( "nltk_tokenization", "count", False, False, 1, 1, 7)=0.09748633527356529
eval( "nltk_tokenization", "count", False, False, 1, 1, 5)=0.09800387831990245
eval( "nltk_tokenization", "count", False, False, 1, 1, 6)=0.09784149321571817
```





```
eval( "space_punk_tokenization", "count", False, False, 0.04, 0.07, 6)=0.120846055858826
eval( "space_punk_tokenization", "count", False, False, .05, .07, 6)=0.12119123673526033
eval( "space_punk_tokenization", "count", False, False, .05, .07, 6.3)=0.12090685535010048

eval( "space_punk_tokenization", "count", False, False, .6, .6, 6)=0.1328096951481551
eval( "space_punk_tokenization", "count", False, False, .7, .7, 9)=0.13255759713493448
eval( "space_punk_tokenization", "count", False, False, .7, .7, 5)=0.13351961590452624
```



```
eval( "space_punk_tokenization", "count", True, False, .01, .01, 6)=0.1702661375318209
eval( "space_punk_tokenization", "count", True, False, .01, .01, 5)=0.1702944397093868
eval( "space_punk_tokenization", "count", True, False, .1, .1, 4)=0.17046565891488336

eval( "space_punk_tokenization", "count", True, False, .8, .8, 9)=0.1811179981526148
eval( "space_punk_tokenization", "count", True, False, .7, .7, 9)=0.18210629850721272
eval( "space_punk_tokenization", "count", True, False, .7, .7, 8)=0.18117506829007374
```



```
eval( "space_punk_tokenization", "count", True, True, .03, .01, 0.08)=0.19587898970966247
eval( "space_punk_tokenization", "count", True, True, .04, .01, 0.08)=0.19504341183513155
eval( "space_punk_tokenization", "count", True, True, .02, .02, .135)=0.19511153916057217

eval( "space_punk_tokenization", "count", True, True, .8, .8, 9)=0.20523882798447549
eval( "space_punk_tokenization", "count", True, True, .7, .7, 9)=0.20505891595199038
eval( "space_punk_tokenization", "count", True, True, .8, .8, 10)=0.20518648691530994
```









```
eval( "space_punk_tokenization", "tf_idf", False, False, .2, .2, 9.5)=0.21022354440060112
eval( "space_punk_tokenization", "tf_idf", False, False, .2, .2, 9.6)=0.21011504763075045
eval( "space_punk_tokenization", "tf_idf", False, False, .2, .2, 9.7)=0.21004802842077883

eval( "space_punk_tokenization", "tf_idf", False, False, 5, 6, 100)=0.21593175562706007
eval( "space_punk_tokenization", "tf_idf", False, False, 6, 6, 100)=0.21638593996660763
eval( "space_punk_tokenization", "tf_idf", False, False, 6, 7, 100)=0.2163596784645163
```




















```
eval( "space_punk_tokenization", "tf_idf", True, False, .2, .2, 9.5)=0.21043315544810065
eval( "space_punk_tokenization", "tf_idf", True, False, .2, .2, 9.6, 10)=0.21043845214744733
eval( "space_punk_tokenization", "tf_idf", True, False, .2, .2, 9.7)=0.21077359372563237

eval( "space_punk_tokenization", "tf_idf", True, False, 7, 7, 150)=0.2161148820585441
eval( "space_punk_tokenization", "tf_idf", True, False, 7, 7, 151)=0.216142135433857
eval( "space_punk_tokenization", "tf_idf", True, False, 7, 7, 152)=0.21610963593512728

```



```
eval( "space_punk_tokenization", "tf_idf", True, True, .1, .1, 8.9)=0.21527317884944822
eval( "space_punk_tokenization", "tf_idf", True, True, .1, .1, 9)=0.2152623072477104
eval( "space_punk_tokenization", "tf_idf", True, True, .1, .1, 8.8)=0.21531085568786323

eval( "space_punk_tokenization", "tf_idf", True, True, 7, 7, 150)= 0.22094843112784524
eval( "space_punk_tokenization", "tf_idf", True, True, 7, 7, 151)=0.22078638262359943
eval( "space_punk_tokenization", "tf_idf", True, True, 7, 7, 152)=0.2208218424059815
```

